# Inflow Hydrographs

__Upstream catchments__

1|upstream = R19 Upstream

2|upstream = R23 Upstream - R8 Downstream

3|upstream = R8 Upstream

__SA Inflow areas__

1|drain = R9 Upstream

2|drain = R30 Upstream - R13 Upstream

3|drain = R14 Downstream - (R23 Downstream + R19 Upstream + R45 Downstream)

4|drain = Outflow - R30 Upstream

5|drain = R45 Upstream

In [1]:
import pandas as pd
import os

DATA_PATH = f"./output/2024/"

critical_event = {
    "50p": {"tp": 4, "dur": 9},
    "20p": {"tp": 4, "dur": 9},
    "10p": {"tp": 5, "dur": 6},
    "5p": {"tp": 8, "dur": 6},
    "2p": {"tp": 3, "dur": 4.5},
    "1p": {"tp": 5, "dur": 4.5},
    "0p05": {"tp": 5, "dur": 4.5}
}

aep_map = {
    "50p": "50",
    "20p": "20",
    "10p": "10",
    "5p": "5",
    "2p": "2",
    "1p": "1",
    "0p05": "1in2000"
}

dur_map = {
    4.5: "4_5hour",
    6: "6hour", 
    9: "9hour"
}

def df_file(storm, aep):
    event = critical_event[aep]
    for i in range(0, 4):
        f = f"{DATA_PATH}{storm}_ aep{aep_map[aep]}_du{dur_map[event['dur']]}tp{event['tp'] + (10 * i)}.csv"
        if os.path.exists(f):
            return f
    raise FileNotFoundError(f"File not found for {aep}")

def load_event(storm, aep):
    event = df_file(storm, aep)
    return pd.read_csv(event, header=2)

def generate_time_series(df, location):
    series = None
    operation = None
    for key in location:
        if series is None:
            series = df[key]
            continue

        if key == "-":
            operation = "-"
            continue
        if key == "+":
            operation = "+"
            continue

        if operation == "+":
            series += df[key]   
        if operation == "-":
            series -= df[key]
    return series

In [2]:
flow_functions = {
    "1|upstream": [" Reach R19 - Upstream"],
    "2|upstream": [" Reach R23 - Upstream", "-",  " Reach R8 - Downstream"],
    "3|upstream": [" Reach R8 - Upstream"],
    "1|drain": [" Reach R9 - Upstream"],
    "2|drain": [" Reach R30 - Upstream", "-", " Reach R13 - Upstream"],
    "3|drain": [" Reach R14 - Downstream", "-", " Reach R23 - Downstream", "-", " Reach R19 - Upstream", "-", " Reach R45 - Downstream"],
    "4|drain": ["Calculated hydrograph:", "-", " Reach R30 - Upstream"],
    "5|drain": [" Reach R45 - Upstream"]
}

storm = "24013_postdev_catchment"
aep = "1p"

Write it out as a TS1 file. 

In [3]:
df = load_event(storm, aep)
columns = ["Time (min)"] + list(flow_functions.keys())
data_out = {}
for col in columns:
    if col == "Time (min)":
        data_out[col] = [v * 60. for v in df["Time (hrs)"]]
    else:
        data_out[col] = generate_time_series(df, flow_functions[col]).to_list()
df_out = pd.DataFrame(data_out)

In [6]:
header = f"""! Generated by Genius
{df_out.shape[1] - 1}, {df_out.shape[0]}
Start_Index, {",".join(["1" for _ in range(df_out.shape[1] - 1)])}
End_Index, {",".join([str(df_out.shape[0]) for _ in range(df_out.shape[1] - 1)])}
"""

with open(f'{storm}_{aep}.ts1', 'w', newline='') as f:
    f.writelines(header)
    df_out.to_csv(f, index=False, float_format='%.6g')